## Ejemplo de como funciona nuestro algoritmo de detección de bboxes para obtener el accuracy

In [1]:
coordenadas_labels = [
    (37.53, 84.11, 81.98, 128.56),
    (5.33, 100.77, 49.78, 145.21),
    (41.97, 181.82, 86.42, 224.0),
    (99.71, 154.06, 144.16, 198.51)
]


coordenadas_labels_eric = [
    (98, 152, 136, 211),  # Piscina 1
    (9, 105, 39, 140),    # Piscina 2
    (39, 87, 76, 114),     # Piscina 3
]

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

accuracys = []
for coordenada in coordenadas_labels:
	
	max_iou = 0
	for coordenada_eric in coordenadas_labels_eric:
		iou = bb_intersection_over_union(coordenada,coordenada_eric)
		if iou > max_iou:
			max_iou = iou
			#print(max_iou)
	accuracys.append(max_iou)
	
print(accuracys)

print(sum(accuracys)/len(accuracys))



[0.5150790106513401, 0.5403709385023008, 0, 0.6251975379139997]
0.4201618717669101


### HSV Canny

In [5]:
import load_hsvANDcanny
import load_clasification

hsv_and_canny_labels = load_hsvANDcanny.hsvCanny(0)
images_with_pools, images_without_pools = load_clasification.clasification()


def convert_oficial_labels(images_with_pools, images_without_pools):
    all_filenames = sorted(list(images_with_pools.keys()) + sorted(images_without_pools))
    oficial_labels = {}
    for filename in all_filenames:
        if filename in images_with_pools:
            oficial_labels[filename] = images_with_pools[filename]
        else:
            oficial_labels[filename] = None  
    return oficial_labels

oficial_labels = convert_oficial_labels(images_with_pools,images_without_pools)

In [6]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def hsv_and_Canny_sin_procesar(oficial_labels, hsv_and_mask_labels):
    #Inicializamos el diccionario de accuracys
    accuracy_hsv = {}
    accuracy_hsv_unicamente_piscinas = {}

    for item in oficial_labels.keys():
        accuracy_hsv[item] = None
        accuracy_hsv_unicamente_piscinas[item] = None
    for item, labels in oficial_labels.items(): #iteramos sobre todo el diccionario de piscinas, por item y labels
        if labels != None:
            accuracys = []
            for hsv_label in hsv_and_mask_labels[item]: #miramos los labels encontrados en la piscina en concreto de hsv
                max_iou = 0
                for label in labels:
                    iou = bb_intersection_over_union(label,hsv_label)
                    if iou > max_iou:
                        max_iou = iou
                        #print(max_iou)
                accuracys.append(max_iou)
            #calculamos el promedio de accuracy en cada iteración
            if (len(accuracys) > 0):
                accuracy_hsv[item] = (sum(accuracys))/len(accuracys)
                accuracy_hsv_unicamente_piscinas[item] = (sum(accuracys))/len(accuracys)
            else:
                accuracy_hsv[item] = sum(accuracys)
                accuracy_hsv_unicamente_piscinas[item] = sum(accuracys)

        else:
            accuracy_hsv_unicamente_piscinas[item] = 0
            if (len(hsv_and_mask_labels[item]) == 0):
                accuracy_hsv[item] = 1
            else:
                accuracy_hsv[item] = 0


    print("ACCURACY DE TODAS LAS IMAGENES")
    print("Si no hay piscina y no se ha detectado piscina, acc=100%, si no hay piscina y se ha detectado piscina, acc=0%")   
    promedio_acc_hsv = sum(accuracy_hsv.values())
    cantidad_imagenes = len(accuracy_hsv)
    print("Sumatorio accuracys: " + str(promedio_acc_hsv))
    print("cantidad de imagenes: " + str(cantidad_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_hsv/cantidad_imagenes))
    print(accuracy_hsv)
    print("------------------------------------------")
    print("ACCURACY DE IMAGENES SOLO PISCINAS")
    promedio_acc_hsv_unicamente_piscinas =  sum(accuracy_hsv_unicamente_piscinas.values())
    cantidad_imagenes = sum(1 for valor in accuracy_hsv_unicamente_piscinas.values() if valor != 0)
    print("Sumatorio accuracys: " + str(promedio_acc_hsv_unicamente_piscinas))
    print("cantidad de imagenes: " + str(cantidad_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_hsv_unicamente_piscinas/cantidad_imagenes))

HSV CANNY PREPROCESADO

In [1]:
import load_hsvANDcanny
import load_clasification

hsv_and_canny_labels = load_hsvANDcanny.hsvCanny(1)
images_with_pools, images_without_pools = load_clasification.clasification()


def convert_oficial_labels(images_with_pools, images_without_pools):
    all_filenames = sorted(list(images_with_pools.keys()) + sorted(images_without_pools))
    oficial_labels = {}
    for filename in all_filenames:
        if filename in images_with_pools:
            oficial_labels[filename] = images_with_pools[filename]
        else:
            oficial_labels[filename] = None  
    return oficial_labels

oficial_labels = convert_oficial_labels(images_with_pools,images_without_pools)


In [4]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def hsv_and_Canny(oficial_labels, hsv_and_mask_labels):
    #Inicializamos el diccionario de accuracys
    accuracy_hsv = {}
    accuracy_hsv_unicamente_piscinas = {}

    for item in oficial_labels.keys():
        accuracy_hsv[item] = None
        accuracy_hsv_unicamente_piscinas[item] = None
    for item, labels in oficial_labels.items(): #iteramos sobre todo el diccionario de piscinas, por item y labels
        if labels != None:
            accuracys = []
            for hsv_label in hsv_and_mask_labels[item]: #miramos los labels encontrados en la piscina en concreto de hsv
                max_iou = 0
                for label in labels:
                    iou = bb_intersection_over_union(label,hsv_label)
                    if iou > max_iou:
                        max_iou = iou
                        #print(max_iou)
                accuracys.append(max_iou)
            #calculamos el promedio de accuracy en cada iteración
            if (len(accuracys) > 0):
                accuracy_hsv[item] = (sum(accuracys))/len(accuracys)
                accuracy_hsv_unicamente_piscinas[item] = (sum(accuracys))/len(accuracys)
            else:
                accuracy_hsv[item] = sum(accuracys)
                accuracy_hsv_unicamente_piscinas[item] = sum(accuracys)

        else:
            accuracy_hsv_unicamente_piscinas[item] = 0
            if (len(hsv_and_mask_labels[item]) == 0):
                accuracy_hsv[item] = 1
            else:
                accuracy_hsv[item] = 0


    print("ACCURACY DE TODAS LAS IMAGENES")
    print("Si no hay piscina y no se ha detectado piscina, acc=100%, si no hay piscina y se ha detectado piscina, acc=0%")   
    promedio_acc_hsv = sum(accuracy_hsv.values())
    cantidad_imagenes = len(accuracy_hsv)
    print("Sumatorio accuracys: " + str(promedio_acc_hsv))
    print("cantidad de imagenes: " + str(cantidad_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_hsv/cantidad_imagenes))
    print(accuracy_hsv)
    print("------------------------------------------")
    print("ACCURACY DE IMAGENES SOLO PISCINAS")
    promedio_acc_hsv_unicamente_piscinas =  sum(accuracy_hsv_unicamente_piscinas.values())
    cantidad_imagenes = sum(1 for valor in accuracy_hsv_unicamente_piscinas.values() if valor != 0)
    print("Sumatorio accuracys: " + str(promedio_acc_hsv_unicamente_piscinas))
    print("cantidad de imagenes: " + str(cantidad_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_hsv_unicamente_piscinas/cantidad_imagenes))

In [3]:
hsv_and_Canny(oficial_labels,hsv_and_canny_labels)

ACCURACY DE TODAS LAS IMAGENES
Si no hay piscina y no se ha detectado piscina, acc=100%, si no hay piscina y se ha detectado piscina, acc=0%
Sumatorio accuracys: 2124.282375393674
cantidad de imagenes: 3748
Accuracy promedio total: 0.56677758148177
{'000000000.jpg': 1, '000000001.jpg': 1, '000000002.jpg': 1, '000000003.jpg': 1, '000000004.jpg': 1, '000000005.jpg': 1, '000000006.jpg': 1, '000000007.jpg': 1, '000000008.jpg': 1, '000000009.jpg': 1, '000000010.jpg': 1, '000000011.jpg': 1, '000000012.jpg': 0, '000000013.jpg': 0, '000000014.jpg': 0.5419248112226172, '000000015.jpg': 0.6475534383993599, '000000016.jpg': 1, '000000017.jpg': 0.5828957509395081, '000000018.jpg': 0, '000000019.jpg': 0, '000000020.jpg': 0.5029179291742296, '000000021.jpg': 0, '000000022.jpg': 0.5448218165026003, '000000023.jpg': 0.7002335544197106, '000000024.jpg': 0, '000000025.jpg': 1, '000000026.jpg': 0, '000000027.jpg': 1, '000000028.jpg': 0, '000000029.jpg': 0, '000000030.jpg': 0, '000000031.jpg': 1, '0000000

### HSV Mask

In [1]:
import load_hsvMask
import load_clasification

hsv_and_mask_labels = load_hsvMask.hsvMask(0)
images_with_pools, images_without_pools = load_clasification.clasification()


def convert_oficial_labels(images_with_pools, images_without_pools):
    all_filenames = sorted(list(images_with_pools.keys()) + sorted(images_without_pools))
    oficial_labels = {}
    for filename in all_filenames:
        if filename in images_with_pools:
            oficial_labels[filename] = images_with_pools[filename]
        else:
            oficial_labels[filename] = None  
    return oficial_labels

oficial_labels = convert_oficial_labels(images_with_pools,images_without_pools)


In [2]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def hsvMask(oficial_labels, hsv_and_mask_labels):
    #Inicializamos el diccionario de accuracys
    accuracy_hsv = {}
    accuracy_hsv_unicamente_piscinas = {}

    for item in oficial_labels.keys():
        accuracy_hsv[item] = None
        accuracy_hsv_unicamente_piscinas[item] = None
    for item, labels in oficial_labels.items(): #iteramos sobre todo el diccionario de piscinas, por item y labels
        if labels != None:
            accuracys = []
            for hsv_label in hsv_and_mask_labels[item]: #miramos los labels encontrados en la piscina en concreto de hsv
                max_iou = 0
                for label in labels:
                    iou = bb_intersection_over_union(label,hsv_label)
                    if iou > max_iou:
                        max_iou = iou
                        #print(max_iou)
                accuracys.append(max_iou)
            #calculamos el promedio de accuracy en cada iteración
            if (len(accuracys) > 0):
                accuracy_hsv[item] = (sum(accuracys))/len(accuracys)
                accuracy_hsv_unicamente_piscinas[item] = (sum(accuracys))/len(accuracys)
            else:
                accuracy_hsv[item] = sum(accuracys)
                accuracy_hsv_unicamente_piscinas[item] = sum(accuracys)

        else:
            accuracy_hsv_unicamente_piscinas[item] = 0
            if (len(hsv_and_mask_labels[item]) == 0):
                accuracy_hsv[item] = 1
            else:
                accuracy_hsv[item] = 0


    print("ACCURACY DE TODAS LAS IMAGENES")
    print("Si no hay piscina y no se ha detectado piscina, acc=100%, si no hay piscina y se ha detectado piscina, acc=0%")   
    promedio_acc_hsv = sum(accuracy_hsv.values())
    cantidad_imagenes = len(accuracy_hsv)
    print("Sumatorio accuracys: " + str(promedio_acc_hsv))
    print("cantidad de imagenes: " + str(cantidad_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_hsv/cantidad_imagenes))
    print(accuracy_hsv)
    print("------------------------------------------")
    print("ACCURACY DE IMAGENES SOLO PISCINAS")
    promedio_acc_hsv_unicamente_piscinas =  sum(accuracy_hsv_unicamente_piscinas.values())
    cantidad_imagenes = sum(1 for valor in accuracy_hsv_unicamente_piscinas.values() if valor != 0)
    print("Sumatorio accuracys: " + str(promedio_acc_hsv_unicamente_piscinas))
    print("cantidad de imagenes: " + str(cantidad_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_hsv_unicamente_piscinas/cantidad_imagenes))

In [3]:
hsvMask(oficial_labels,hsv_and_mask_labels)

ACCURACY DE TODAS LAS IMAGENES
Si no hay piscina y no se ha detectado piscina, acc=100%, si no hay piscina y se ha detectado piscina, acc=0%
Sumatorio accuracys: 2447.4139682745645
cantidad de imagenes: 3748
Accuracy promedio total: 0.6529919872664259
{'000000000.jpg': 1, '000000001.jpg': 1, '000000002.jpg': 1, '000000003.jpg': 1, '000000004.jpg': 1, '000000005.jpg': 1, '000000006.jpg': 1, '000000007.jpg': 1, '000000008.jpg': 1, '000000009.jpg': 1, '000000010.jpg': 1, '000000011.jpg': 1, '000000012.jpg': 0.7328063221837083, '000000013.jpg': 0.7328063221837083, '000000014.jpg': 0.6445991823333667, '000000015.jpg': 0.6420301963846182, '000000016.jpg': 1, '000000017.jpg': 0.5632618230726819, '000000018.jpg': 0.6074846925886866, '000000019.jpg': 0.45798054281956146, '000000020.jpg': 0.5193004225656226, '000000021.jpg': 0.4753390220585632, '000000022.jpg': 0.5600428530136805, '000000023.jpg': 0.6461077815007057, '000000024.jpg': 0, '000000025.jpg': 1, '000000026.jpg': 0.7857061394972041, '0

HSV MASK PREPROCESADO

In [4]:
import load_hsvMask
hsv_and_mask_labels = load_hsvMask.hsvMask(1)

In [5]:
hsvMask(oficial_labels,hsv_and_mask_labels)

ACCURACY DE TODAS LAS IMAGENES
Si no hay piscina y no se ha detectado piscina, acc=100%, si no hay piscina y se ha detectado piscina, acc=0%
Sumatorio accuracys: 2357.9918803243813
cantidad de imagenes: 3748
Accuracy promedio total: 0.6291333725518626
{'000000000.jpg': 1, '000000001.jpg': 1, '000000002.jpg': 1, '000000003.jpg': 1, '000000004.jpg': 1, '000000005.jpg': 1, '000000006.jpg': 1, '000000007.jpg': 1, '000000008.jpg': 1, '000000009.jpg': 1, '000000010.jpg': 1, '000000011.jpg': 1, '000000012.jpg': 0.7895590559416691, '000000013.jpg': 0.7881306725278358, '000000014.jpg': 0.6504156220747213, '000000015.jpg': 0.5992320574478793, '000000016.jpg': 1, '000000017.jpg': 0.5805922660093171, '000000018.jpg': 0.6241700049576538, '000000019.jpg': 0.4683220267158824, '000000020.jpg': 0.4628843276311484, '000000021.jpg': 0.4267665949649338, '000000022.jpg': 0.5377851118278004, '000000023.jpg': 0.6841617124970352, '000000024.jpg': 0, '000000025.jpg': 1, '000000026.jpg': 0.7379617032820548, '00

### YOLOv8 SIN MODIFICAR ACCURACY

In [12]:
import load_ultralytics
import load_clasification

#Primer parametro epochs (10 o 100), segundo parámetro modificador, si usamos las imagenes modificadas entonces es True
ultralytics_labels = load_ultralytics.load_ultralytics_dicc(10,False)
images_with_pools, images_without_pools = load_clasification.clasification()


def convert_oficial_labels(images_with_pools, images_without_pools):
    all_filenames = sorted(list(images_with_pools.keys()) + sorted(images_without_pools))
    oficial_labels = {}
    for filename in all_filenames:
        if filename in images_with_pools:
            oficial_labels[filename] = images_with_pools[filename]
        else:
            oficial_labels[filename] = None  
    return oficial_labels

oficial_labels = convert_oficial_labels(images_with_pools,images_without_pools)


0: 224x224 (no detections), 24.8ms
Speed: 1.0ms preprocess, 24.8ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 22.8ms
Speed: 0.5ms preprocess, 22.8ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 22.3ms
Speed: 0.5ms preprocess, 22.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 22.3ms
Speed: 0.5ms preprocess, 22.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 23.8ms
Speed: 0.5ms preprocess, 23.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 3 pools, 22.3ms
Speed: 0.5ms preprocess, 22.3ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 21.8ms
Speed: 0.5ms preprocess, 21.8ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 3 pools, 22.3ms
Speed: 0.5ms preprocess, 22.3ms inference, 0.5ms 

In [15]:
def convert_to_tuples(coords_list):
    if coords_list is None:
        return []
    result = []
    for sublist in coords_list:
        if isinstance(sublist, list) and len(sublist) == 1:
            coords = sublist[0]
            result.append((int(coords[0]), int(coords[1]), int(coords[2]), int(coords[3])))
    return result



def Yolov8_sin_modificar(oficial_labels,ultralytics_labels):
    # Procesar el diccionario para transformar las listas de listas en listas de tuplas
    processed_dict = {}

    for key, value in ultralytics_labels.items():
        # Convertir las listas de coordenadas en listas de tuplas
        processed_dict[key] = convert_to_tuples(value)

    
    #Inicializamos el diccionario de accuracys
    accuracy_ultralytics = {}
    accuracy_ultralytics_unicamente_piscinas = {}

    for item in processed_dict.keys():
        accuracy_ultralytics[item] = None
        accuracy_ultralytics_unicamente_piscinas[item] = None

    claves_a_eliminar = []

    # Iteramos sobre las claves del diccionario grande
    for clave in oficial_labels:
        if clave not in processed_dict:  # Verificamos si la clave no está en el diccionario pequeño
            claves_a_eliminar.append(clave)  # Agregamos la clave a la lista de claves a eliminar

    # Eliminamos las claves del diccionario grande que no están en el diccionario pequeño
    for clave in claves_a_eliminar:
        del oficial_labels[clave]

    for clave, valor in oficial_labels.items():
        if valor == 0:  # Verificar si el valor es 0
            oficial_labels[clave] = None

    for item, labels in oficial_labels.items(): #iteramos sobre todo el diccionario de piscinas, por item y labels
        if labels != None:
            accuracys = []
            for hsv_label in processed_dict[item]: #miramos los labels encontrados en la piscina en concreto de hsv
                max_iou = 0
                for label in labels:
                    iou = bb_intersection_over_union(label,hsv_label)
                    if iou > max_iou:
                        max_iou = iou
                        #print(max_iou)
                accuracys.append(max_iou)
            #calculamos el promedio de accuracy en cada iteración
            if (len(accuracys) > 0):
                accuracy_ultralytics[item] = (sum(accuracys))/len(accuracys)
                accuracy_ultralytics_unicamente_piscinas[item] = (sum(accuracys))/len(accuracys)
            else:
                accuracy_ultralytics[item] = sum(accuracys)
                accuracy_ultralytics_unicamente_piscinas[item] = sum(accuracys)

        else:
            accuracy_ultralytics_unicamente_piscinas[item] = 0
            if (len(processed_dict[item]) == 0):
                accuracy_ultralytics[item] = 1
            else:
                accuracy_ultralytics[item] = 0

    total_imagenes = 600
    total_imagenes_unicamente_con_piscinas = 300

    promedio_acc_ultralytics = sum(accuracy_ultralytics.values())
    print("Sumatorio accuracys: " + str(promedio_acc_ultralytics))
    print("cantidad de imagenes totales: " + str(total_imagenes))
    print("Accuracy promedio total: "+ str(promedio_acc_ultralytics/total_imagenes))
    print("-------------------------------------------------------------------------")
    promedio_acc_ultralytics_unicamente_piscinas = sum(accuracy_ultralytics_unicamente_piscinas.values())
    print("Sumatorio accuracys: " + str(promedio_acc_ultralytics_unicamente_piscinas))
    print("cantidad de imagenes unicamente piscinas: " + str(total_imagenes_unicamente_con_piscinas))
    print("Accuracy promedio total: "+ str(promedio_acc_ultralytics_unicamente_piscinas/total_imagenes_unicamente_con_piscinas))

In [16]:
Yolov8_sin_modificar(oficial_labels,ultralytics_labels)

Sumatorio accuracys: 475.25172607441135
cantidad de imagenes totales: 600
Accuracy promedio total: 0.7920862101240189
-------------------------------------------------------------------------
Sumatorio accuracys: 180.25172607441135
cantidad de imagenes unicamente piscinas: 300
Accuracy promedio total: 0.6008390869147046
